## Visualization / Mapping Functions

In [14]:
# color coding function
def color_code_fn(school_type):
    if school_type == "elementary school":
        color_code = [255,0,125,255]
    elif school_type == "middle school":
        color_code = [255,0,0,255]  
    elif school_type == "high school":
        color_code = [125,0,255,255] 
    else:
        color_code = [125,0,255,255] 
    return color_code

# circle diameter function
def circle_diameter_fn(max_diameter, time_amount, late_time_array):
    d = (max_diameter*(time_amount+30))/max(late_time_array)
    return d

def map_all_schools_fn(school):
    circle_location = school['geocode']['location']
    pop_up_description = {"title" : school['School'], "content" : school['geocode']['address']}
    circle_format = {"type": "esriSMS","style": "esriSMSCircle","color": [0,0,0,125],"size": 4,
                         "angle": 0,"xoffset": 0,"yoffset": 0,
                         "outline": {"color": [0,0,0,255],"width": 0}}
    return [circle_location, pop_up_description, circle_format]

def map_late_schools_fn(school, d, color_code):
    circle_location = school['geocode']['location']
    pop_up_description = {"title" : school['School'], "content" : "Late: "+str(school['Late Time'])+" minutes"}
    circle_format = {"type": "esriSMS","style": "esriSMSCircle","color": color_code,
                     "size": d,"angle": 0,"xoffset": 0,"yoffset": 0,
                     "outline": {"color": [0,0,0,255],"width": 1}}
    return [circle_location, pop_up_description, circle_format]

## Data analysis / relational functions

In [12]:
def todays_late_buses_fn(late_bus_df):
    late_bus_df = pd.read_pickle('scraping_bus_data/bus_data.pk1')
    late_bus_df1  = late_bus_df.loc[late_bus_df['Day'] == str(7)]#str(now.day)]
    late_bus_df2  = late_bus_df1.loc[late_bus_df1['To/From'] == 'to']
    late_bus_df3  = late_bus_df2.loc[late_bus_df2['Data Taken Hour'] == 7]
    select_late_bus_df = late_bus_df3

    # get school info for schools with late buses    
    late_schools_df = school_df.loc[school_df.get('School').isin(late_bus_df3.get('School'))]
    
    return [select_late_bus_df, late_schools_df]

def late_minutes_per_school_fn(select_late_bus_df, late_schools_df):
    # number of schools affected
    total_rows = late_schools_df.shape[0]#int(late_schools.size/8)
    print(total_rows)
    
    # initialize late_time array
    late_time = (total_rows)*[0]
    i = 0
    
    # add minute late column to school df
    for index_schools, row_schools in late_schools_df.iterrows():
        for index_buses, row_buses in select_late_bus_df.iterrows():
            if row_schools['School'] == row_buses['School']:
                late_time[i] += row_buses['Time']
        i+=1
    late_schools_df['Late Time'] = late_time
    return late_schools_df

# Map of today's late schools

In [16]:
import datetime
import pandas as pd
import numpy as np
from datascience import *
import random
from arcgis.gis import GIS

now = datetime.datetime.now()

# read in schools table
school_df = pd.read_pickle('geocoding_schools/cleaned_school_list.pk1')

#from arcgis.geocoding import geocoding
my_gis = GIS()
seattle = my_gis.map("Seattle, WA")

# map all schools
for index, school in school_df.iterrows():
    try: 
        [circle_location, pop_up_description, circle_format] = map_all_schools_fn(school)
        seattle.draw(circle_location, pop_up_description, circle_format)
    # some schools don't have an address in the table
    except:
        pass  
    
# get today's late bus data
late_bus_df = pd.read_pickle('scraping_bus_data/bus_data.pk1')
[select_late_bus_df, late_schools_df] = todays_late_buses_fn(late_bus_df)

#calculate the total bus late time for each school
late_schools_df = late_minutes_per_school_fn(select_late_bus_df, late_schools_df)

# map late schools
for index, school in late_schools_df.iterrows():
    color_code = color_code_fn(school['Type'])
    d = circle_diameter_fn(max_diameter = 8, time_amount = school['Late Time'], 
                           late_time_array = late_schools_df['Late Time'])
    [circle_location, pop_up_description, circle_format] = map_late_schools_fn(school, d, color_code)
    seattle.draw(circle_location, pop_up_description, circle_format)
    
seattle

10


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


MapView(layout=Layout(height='400px', width='100%'))

# All Days